# **MIP3_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand."""


In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
   - x_ij = Power output of power unit i in period j
   - y_ij = Status of power plant i in period j (binary, 1 if power plant is turned on, 0 otherwise)
   - s_i  = Startup cost for power unit i 
   - d_i  = Shutdown cost for power unit i
   - v_ij = Variable cost for power unit i in period j
   - f_i  = Fixed cost for power unit i 
   - u_i  = Upper bound for power output of power unit i
   - l_i  = Lower bound for power output of power unit i
   - mP_i = Maximum power increment for power unit i
   - mD_i = Maximum power decrement for power unit i

2. Parameters:
   - D_j = Total power demand in period j
   - S_i = Startup costs of power unit i (i ranges from 1 to 6)
   - T_i = Shutdown costs of power unit i (i ranges from 1 to 6)
   - F_i = Fixed costs of power unit i (i ranges from 1 to 6)
   - VC_ij = Variable costs for power unit i at time period j
   - UB_i = Upper bound of power output for power unit i
   - LB_i = Lower bound of power output for power uni

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
print(response2.choices[0].message.content)

Objective function:

Minimize
∑(over i from 1 to 6) ∑(over j from 1 to 15) [(s_i * S_i) + (d_i * T_i) + y_ij*(F_i + f_i + (x_ij * VC_ij))]

Where:
- s_i = {1, if power plant i starts up at period j; 0, otherwise}
- d_i = {1, if power plant i shuts down at period j; 0, otherwise}
- y_ij = {1, if power plant i is on at period j; 0, otherwise}


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [10]:
print(response3.choices[0].message.content)

Constraints:

1. Power output constraints:
   - The power output of each power unit i in each period j must be within their lower and upper bounds.
      
      LB_i <= x_ij <= UB_i  for all i in [1,6] and for all j in [1,15]
   
2. Maximum power increment/decrement constraints:
   - The change in power output from one period to the next cannot exceed the maximum power increment or decrement for each power unit.
      
      x_ij - x_i(j-1) <= PI_i   for all i in [1,6] and for all j in [2,15]
      x_i(j-1) - x_ij <= PD_i   for all i in [1,6] and for all j in [2,15]
   
3. Power unit status constraints:
   - If the power unit i is on in period j, it must have been either on in the previous period or must have started up in this period. Similarly, if it is off, it must have been either off in the previous period or must have shut down in this period.
      
      y_ij - y_i(j-1) <= s_i   for all i in [1,6] and for all j in [2,15]
      y_i(j-1) - y_ij <= d_i   for all i in [1,6] and for

### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [12]:
Markdown(response4.choices[0].message.content)

Below is the sample Python code using the Pyomo library:

```python
from pyomo.environ import *

model = ConcreteModel()

# Sets
model.I = RangeSet(1, 6)  # Power units
model.J = RangeSet(1, 15)  # Time periods

# Parameters(Sample data used)
model.LB = Param(model.I, default = 20)
model.UB = Param(model.I, default = 50)
model.D = Param(model.J, default = 200)
model.PI = Param(model.I, default = 5)
model.PD = Param(model.I, default = 5)
model.S = Param(model.I, default = 3)
model.T = Param(model.I, default = 2)
model.F = Param(model.I, default = 10)
model.VC = Param(model.I, model.J, default = 1)
model.RD = Param(model.J, default = 220)

# Variables
model.x = Var(model.I, model.J, bounds=(0,None))
model.y = Var(model.I, model.J, within=Binary)
model.s = Var(model.I, model.J, within=Binary)
model.d = Var(model.I, model.J, within=Binary)

# Objective
model.obj = Objective(expr = sum(sum((model.s[i,j]*model.S[i] + model.d[i,j]*model.T[i] + model.y[i,j]*(model.F[i] + model.VC[i,j]*model.x[i,j])) 
for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.PowerOutput = Constraint(model.I, model.J, rule=lambda model, i, j:model.LB[i] <= model.x[i,j] <= model.UB[i])

def Increment_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j] - model.x[i,j-1] <= model.PI[i]
model.Increment = Constraint(model.I, model.J, rule=Increment_rule)

def Decrement_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j-1] - model.x[i,j] <= model.PD[i]
model.Decrement = Constraint(model.I, model.J, rule=Decrement_rule)

def Status_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.y[i,j] - model.y[i,j-1] <= model.s[i,j], model.y[i,j-1] - model.y[i,j] <= model.d[i,j]
model.Status = Constraint(model.I, model.J, rule=Status_rule)

model.TotalDemand = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.D[j])

model.Security = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.RD[j])

# Solve
SolverFactory('glpk').solve(model)
```
This script creates a model to solve the optimization problem. I have used GLPK as a linear solver in this case. The binary variables are the on/off status, startup, and shutdown decisions of power units. The continuous variables are the power outputs of power units. The objective function minimizes the total cost and the constraints ensure the requirements are fulfilled. Sample data are entered as default parameter values.

### **Run the code Generated by GPT4**

In [15]:
from pyomo.environ import *

model = ConcreteModel()

# Sets
model.I = RangeSet(1, 6)  # Power units
model.J = RangeSet(1, 15)  # Time periods

# Parameters(Sample data used)
model.LB = Param(model.I, default = 20)
model.UB = Param(model.I, default = 50)
model.D = Param(model.J, default = 200)
model.PI = Param(model.I, default = 5)
model.PD = Param(model.I, default = 5)
model.S = Param(model.I, default = 3)
model.T = Param(model.I, default = 2)
model.F = Param(model.I, default = 10)
model.VC = Param(model.I, model.J, default = 1)
model.RD = Param(model.J, default = 220)

# Variables
model.x = Var(model.I, model.J, bounds=(0,None))
model.y = Var(model.I, model.J, within=Binary)
model.s = Var(model.I, model.J, within=Binary)
model.d = Var(model.I, model.J, within=Binary)

# Objective
model.obj = Objective(expr = sum(sum((model.s[i,j]*model.S[i] + model.d[i,j]*model.T[i] + model.y[i,j]*(model.F[i] + model.VC[i,j]*model.x[i,j]))
for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.PowerOutput = Constraint(model.I, model.J, rule=lambda model, i, j:model.LB[i] <= model.x[i,j] <= model.UB[i])

def Increment_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j] - model.x[i,j-1] <= model.PI[i]
model.Increment = Constraint(model.I, model.J, rule=Increment_rule)

def Decrement_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j-1] - model.x[i,j] <= model.PD[i]
model.Decrement = Constraint(model.I, model.J, rule=Decrement_rule)

def Status_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.y[i,j] - model.y[i,j-1] <= model.s[i,j], model.y[i,j-1] - model.y[i,j] <= model.d[i,j]
model.Status = Constraint(model.I, model.J, rule=Status_rule)

model.TotalDemand = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.D[j])

model.Security = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.RD[j])

# Solve
SolverFactory('glpk').solve(model)

ERROR:pyomo.core:Rule failed when generating expression for Constraint PowerOutput with index (1, 1):
PyomoException: Cannot convert non-constant Pyomo expression (20  <=  x[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.
ERROR:pyomo.core:Constructing component 'PowerOutput' from data=None failed:
    PyomoException: Cannot convert non-constant Pyomo expression (20  <=  x[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...  

PyomoException: Cannot convert non-constant Pyomo expression (20  <=  x[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

In [22]:
from pyomo.environ import *

model = ConcreteModel()

# Sets
model.I = RangeSet(1, 6)  # Power units
model.J = RangeSet(1, 15)  # Time periods

# Parameters(Sample data used)
model.LB = Param(model.I, initialize = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65})
model.UB = Param(model.I, default = {1: 500, 2: 600, 3: 550, 4: 700, 5: 650, 6: 750})
model.D = Param(model.J, default = 200)
model.PI = Param(model.I, default = 5)
model.PD = Param(model.I, default = 5)
model.S = Param(model.I, default = 3)
model.T = Param(model.I, default = 2)
model.F = Param(model.I, default = 10)
model.VC = Param(model.I, model.J, default = 1)
model.RD = Param(model.J, default = 220)

# Variables
model.x = Var(model.I, model.J, bounds=(0,None))
model.y = Var(model.I, model.J, within=Binary)
model.s = Var(model.I, model.J, within=Binary)
model.d = Var(model.I, model.J, within=Binary)

# Objective
model.obj = Objective(expr = sum(sum((model.s[i,j]*model.S[i] + model.d[i,j]*model.T[i] + model.y[i,j]*(model.F[i] + model.VC[i,j]*model.x[i,j]))
for i in model.I) for j in model.J), sense=minimize)

# Constraints
model.PowerOutput = Constraint(model.I, model.J, rule=lambda model, i, j:model.LB[i] <= model.x[i,j])
model.PowerOutput1 = Constraint(model.I, model.J, rule=lambda model, i, j:model.x[i,j] <= model.UB[i])

def Increment_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j] - model.x[i,j-1] <= model.PI[i]
model.Increment = Constraint(model.I, model.J, rule=Increment_rule)

def Decrement_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.x[i,j-1] - model.x[i,j] <= model.PD[i]
model.Decrement = Constraint(model.I, model.J, rule=Decrement_rule)

def Status_rule(model, i, j):
    if j==1:
        return Constraint.Skip
    return model.y[i,j] - model.y[i,j-1] <= model.s[i,j], model.y[i,j-1] - model.y[i,j] <= model.d[i,j]
model.Status = Constraint(model.I, model.J, rule=Status_rule)

model.TotalDemand = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.D[j])

model.Security = Constraint(model.J, rule=lambda model, j:sum(model.x[i,j] for i in model.I) >= model.RD[j])

# Solve
SolverFactory('glpk').solve(model)

ERROR:pyomo.core:Rule failed when generating expression for Constraint Status with index (1, 2):
ValueError: Constraint 'Status[1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519690>, <pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519a50>) contained invalid type, InequalityExpression
ERROR:pyomo.core:Constructing component 'Status' from data=None failed:
    ValueError: Constraint 'Status[1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519690>, <pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519a50>) contained invalid type, InequalityExpression


ValueError: Constraint 'Status[1,2]' does not have a proper value. Constraint expressions expressed as tuples must contain native numeric types or Pyomo NumericValue objects. Tuple (<pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519690>, <pyomo.core.expr.relational_expr.InequalityExpression object at 0x7d3d00519a50>) contained invalid type, InequalityExpression

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**